In [2]:
!pip install -U langchain-community

In [3]:
pip install chromadb

In [4]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

In [7]:
import os
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFaceHub
from langchain.chains import RetrievalQA

# Set Hugging Face API key (Needed for LLMs from Hugging Face Hub)
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "
# Load and preprocess documents
loader = TextLoader("/content/docs.txt")  # Replace with your text file
documents = loader.load()

# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = text_splitter.split_documents(documents)

# Use Hugging Face embeddings (without FAISS)
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = Chroma.from_documents(texts, embedding_model)  # ChromaDB as vector store

# Set up a retriever
retriever = vectorstore.as_retriever()

# Load Hugging Face LLM (Example: Falcon-7B)
llm = HuggingFaceHub(repo_id="tiiuae/falcon-7b-instruct", model_kwargs={"temperature": 0.5, "max_length": 512})



In [8]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

# Define a clean prompt template that includes the 'context' variable
prompt_template = """Answer the following question as concisely as possible.
If you don't know the answer, just say "I don't know."


Context: {context}
Question: {question}
Answer:"""

# Apply the template to ensure cleaner output
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=False,  # Ensures only the answer is returned
    chain_type_kwargs={"prompt": PromptTemplate(template=prompt_template, input_variables=["context", "question"])}
)

In [9]:
def format_response(response, query):
    # Split response by lines
    lines = response.split("\n")

    # Find the line where the question starts
    for i, line in enumerate(lines):
        if line.strip().startswith("Question:"):
            # The answer is expected in the next line
            if i + 1 < len(lines):
                answer = lines[i + 1].replace("Answer:", "").strip()
                return f"Question: {query}\nAnswer: {answer}"

    return f"Question: {query}\nAnswer: Answer not found."



In [10]:
# Run query
query = "What are the applications of RAG?"
response = qa_chain.run(query)

# Format the output
formatted_output = format_response(response, query)
print(formatted_output)  # Prints in the required format

<ipython-input-10-9def09d63df7>:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain.run(query)
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


Question: What are the applications of RAG?
Answer: Information Retrieval, Document Summarization, and Chatbots.
